In [51]:
import re
from random import randrange, shuffle
import torch
from torch import nn
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

-pre-training
    training on known task t known.. 
    Task is not necassarly domain speciifc an pre-training is done on a big coprus, with task_t (characterstic. Large, captures the general properties of the language. focus isn't on the target task)
    as an example; when using word2vec, or other pre-trained models, that have already been trainewd on the general lanuage, and adapting them or soecific taks, (followed by domain adaptation)
    common tasks for pre-training are MLM and NSP to understand the language

-domain adaptation 
    target task doesn't equal task used when training, which would require training on data linked to the target task. (i.e., adaptin to the specitic domain)
    adaptation based on task s, unknown and doesnt' equal t
    final model would be a pre-trained general-domain model, that would later on required fine tuning
    same pre-training tasks

-fine-tuning 
    (in simple words tweaking the hyperparameters of the model to get the best results) fin-tuning approach has a high impact on the results/
    understanding the language


# Exercise 8: Encoder Model

### Toy example based
This code is based on code developed by Dong-Hyun Lee.


Please, read carefully the code, since it could help you in the subsequent programming tasks. While you read the text, try to answer each of the questions that are presented in the comments

In [52]:
# Example of a single review. This review is a modified version of the first positive review of the dataset that we are using
# Note that is a single review
review = ["Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as \"Teachers\"."
        "My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is \"Teachers\"."
         "The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students."
         "When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled  at  High."
         "A classic line: INSPECTOR: I'm here to sack one of your teachers, STUDENT: Welcome to Bromwell High."
         "I expect that many adults of my age think that Bromwell High is far fetched."
         "What a pity that it isn't!"]


In [ ]:
# Importantly, we are adding some special tokens to the vocabulary.
# From now, start thinking what is tok1, tok2, tok3, and tok4 (based on there usage in create_B)
TOK_1 = '[tok1]' # Padding? [PAD]
TOK_2 = '[tok2]' #start of sentence ?? # [CLS]
TOK_3 = '[tok3]' # seperator? [SEP]
TOK_4 = '[tok4]' # [MASK]

# Basic tokenizer - you should use your tokenizer from previous exercises or an improved version of the following code, but you can use the following code as a starting point
def build_vocab_tokenize(reviews):
    sentences = []
    for rev in reviews:
        parts = re.split(r'[.!?]+', rev)
        for part in parts:
            sent = part.strip().lower()
            if not sent:
                continue
            sent = re.sub(r"[^a-z0-9'\s]", "", sent)
            sentences.append(sent)

    all_words = " ".join(sentences).split()
    vocab = sorted(set(all_words))

    tokens_2_index_dict = {TOK_1: 0, TOK_2: 1, TOK_3: 2, TOK_4: 3}

    # The following loops should be familiar to you, since we have been doing this from exercise #2
    init_index = len(tokens_2_index_dict)

    # Create two dictionaries one for mapping index (ids) to tokens and one for tokens to index
    for i, token in enumerate(vocab):
        tokens_2_index_dict[token] = i + init_index

    index_2_token = {}
    for i, token in enumerate(tokens_2_index_dict):
        index_2_token[i] = token

    sentences_2_token_lst = [
        [tokens_2_index_dict[word] for word in sent.split()]
        for sent in sentences
    ]
    vocab_size = len(tokens_2_index_dict)
 
    """for sent in sentences:
        print(sent)"""
    return tokens_2_index_dict,index_2_token, vocab_size, sentences_2_token_lst, sentences

In [54]:
build_vocab_tokenize(review)



({'[tok1]': 0,
  '[tok2]': 1,
  '[tok3]': 2,
  '[tok4]': 3,
  '35': 4,
  'a': 5,
  'about': 6,
  'adults': 7,
  'age': 8,
  'all': 9,
  'and': 10,
  'as': 11,
  'at': 12,
  'believe': 13,
  'bromwell': 14,
  'burn': 15,
  'can': 16,
  'cartoon': 17,
  'classic': 18,
  'closer': 19,
  'comedy': 20,
  'down': 21,
  'episode': 22,
  'expect': 23,
  'far': 24,
  'fetched': 25,
  'financially': 26,
  'here': 27,
  'high': 28,
  "high's": 29,
  'i': 30,
  "i'm": 31,
  'immediately': 32,
  'in': 33,
  'insightful': 34,
  'inspector': 35,
  'is': 36,
  "isn't": 37,
  'it': 38,
  'knew': 39,
  'lead': 40,
  'life': 41,
  'line': 42,
  'many': 43,
  'me': 44,
  'much': 45,
  'my': 46,
  'of': 47,
  'one': 48,
  'other': 49,
  'pathetic': 50,
  'pettiness': 51,
  'pity': 52,
  'pomp': 53,
  'profession': 54,
  'programs': 55,
  'ran': 56,
  'reality': 57,
  'recalled': 58,
  'remind': 59,
  'repeatedly': 60,
  'right': 61,
  'sack': 62,
  'same': 63,
  'satire': 64,
  'saw': 65,
  'school': 66,
 

## Theoretical part 1 - code related

In [ ]:
maxlen_X = 100 # what is X? maximum length per sequence 
size_B = 6 # What is B? batch size??
max_pred_M = 10  # what is M? number of masked tokens

In [56]:


def create_B(maxlen_X=maxlen_X, size_B=size_B, max_pred_M=max_pred_M
             ,sentences=[] , sentences_2_tokens_lst=[], tokens_2_index_dict=[]):
    assert size_B % 2 == 0, "size_B should be even"
    
    lst_B = []
    positive_pair_X = 0
    negative_pair_X = 0
    number_sentences = len(sentences)

    while positive_pair_X != size_B/2 or negative_pair_X != size_B/2: # we want 50% of something positive and 50% of something negative
        tokens_X1_index = randrange(number_sentences)
        tokens_X2_index = randrange(number_sentences)

        tokens_X1 = sentences_2_tokens_lst[tokens_X1_index]
        tokens_X2 = sentences_2_tokens_lst[tokens_X2_index]

        #What are we doing here? WHy do we need to do this?
        # for NSP tasks. differentianting between sentences. would we need to do that if we're not using NSP?
        input_ids_X = [tokens_2_index_dict[TOK_2]] + tokens_X1 + [tokens_2_index_dict[TOK_3]] + tokens_X2 + [tokens_2_index_dict[TOK_3]]

        int_max_number_pred = max(1,int(round(len(input_ids_X) * 0.15)))
        n_pred_M =  min(max_pred_M, int_max_number_pred) # max - 15% of tokens in one STU
        
        # every token can be M? no, preset tokens [CLS] [SEP] are excluded. masking these would
        cand_M_pos = []
        for i, token in enumerate(input_ids_X):
            if token != tokens_2_index_dict[TOK_2] and token != tokens_2_index_dict[TOK_3]:
                cand_M_pos.append(i)

        
        shuffle(cand_M_pos) # Why do we need to shuffle this? to mask random tokens (that are not preset)
        M_tokens = []
        M_pos = []

        #What is this loop iterating? after the shuffle, we're storing the randomly selected tokens for masking, (to use to check prediction), and then masking them with [tok4] [MASK]
        for pos in cand_M_pos[:n_pred_M]:
            M_pos.append(pos)
            M_tokens.append(input_ids_X[pos])
            input_ids_X[pos] = tokens_2_index_dict[TOK_4]


        # Why do we need to pad input_ids_X? 
        #maked sure every input sequence has the same length (Batch, maxlen). padding here is looking at input
        #to ensure all sequenecs are of the same length of training (uniform shape)
        n_pad = maxlen_X - len(input_ids_X)
        input_ids_X.extend([tokens_2_index_dict[TOK_1]] * n_pad)


        
        # What would happen if we do not have the conditional?
        ## n_pred_M is the number of tokens to be masked in the given sequence, and max_pred_M is the maximum number of tokens to max.
        # if our sequence length is too short then the number n_pred_max (we would have less tokens)
        if max_pred_M > n_pred_M:
            n_pad = max_pred_M - n_pred_M
            M_tokens.extend([tokens_2_index_dict[TOK_1]] * n_pad)
            M_pos.extend([tokens_2_index_dict[TOK_1]] * n_pad)

        # What are we verifying with this conditional?
        #for NSP, checks that the sentence are consecutive and that half are positive and half are negative
        if tokens_X1_index + 1 == tokens_X2_index and positive_pair_X < size_B/2:
            lst_B.append([input_ids_X, M_tokens, M_pos, True])
            positive_pair_X += 1

        elif tokens_X1_index + 1 != tokens_X2_index and negative_pair_X < size_B/2:
            lst_B.append([input_ids_X, M_tokens, M_pos, False])
            negative_pair_X += 1

    return lst_B

In [57]:
tokens_2_index_dict,_, _, sentences_2_token_lst, sentences = build_vocab_tokenize(review)
B = create_B(sentences=sentences, sentences_2_tokens_lst= sentences_2_token_lst , tokens_2_index_dict=tokens_2_index_dict)

In [58]:
input_ids_Xs, M_tokens, M_pos, isNext = map(torch.LongTensor, zip(*B))

## Programming Tasks: Implementing an Encoder Model

In [59]:
# You can reuse the `SelfAttention` class from Exercise 6, but you might need to extend it to support masking (C.1.1.v).
class SelfAttention(nn.Module):
    def __init__(self, d_model, d_k):
        super().__init__()
        # define the weight matrices W_Q, W_K, and W_V
        self.model = d_model
        self.d_k = d_k

        # learnable weight matrices
        self.W_Q = nn.Linear(d_model, d_k) #x @ weight + bias
        self.W_K = nn.Linear(d_model, d_k) #x @ weight + bias
        self.W_V = nn.Linear(d_model, d_k) #....


    def forward(self, x, mask=None):
        #vector production
        W_Q = self.W_Q(x)
        W_K = self.W_K(x)
        W_V = self.W_V(x)
         
        #attention score using dot product between queries and keys
        scores = torch.matmul (W_Q, W_K.transpose(-2,-1)) / np.sqrt(self.d_k)

        if mask is not None: 
            #adding minus infinity to masked values (with 0 zero) to nullify after softmax
            #-1e9
            mask = mask.unsqueeze(1)
            scores= scores.masked_fill(mask ==0, float('-inf'))
        W_attention = F.softmax(scores, dim=-1)
        #attention-weighted sum of the value vector
        out = torch.matmul(W_attention, W_V)
        return out

In [60]:
# C.1.1 Multi-Head Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, d_k=None):
        super().__init__()
        self.d_model = d_model
        self.d_k = d_k
        self.n_heads = n_heads


        #concatination (head_1, head_2,... head_n)

        self.heads = nn.ModuleList(
            [SelfAttention(d_model, d_k) for _ in range (n_heads) ]
        )
        #linear projection/ dense layer
        self.proj = nn.Linear(n_heads * d_k, d_model)

    def forward(self, x, mask=None):

        head_output = []

        #calling forward for each self-attention 
        for head in self.heads: 
            out = head(x, mask= mask)
            head_output.append(out)
        concat = torch.cat(head_output, dim =-1)

        return self.proj(concat)
    
    # C.1.2 Add Residual Connection and LayerNorm
class MultiHeadBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_k=None):
        super().__init__()
        self.attn = MultiHeadAttention(d_model, n_heads, d_k )
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        attn_out = self.attn(x, mask)
        # input = x + atten_out residual connection
        return self.norm(x + attn_out)
    
    

In [61]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        residual = x
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return self.norm(residual + x)

In [62]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_k, dim_feedforward):
        super().__init__()
        self.mha_block = MultiHeadBlock(d_model, n_heads, d_k)
        self.ff_block = FeedForward(d_model, dim_feedforward)

    def forward(self, x, mask=None):
        x = self.mha_block(x, mask)
        x = self.ff_block(x)
        return x


In [63]:
class EncoderModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, nhead=4, num_layers=8, dim_ff=128, max_len=100):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, embed_dim)
        self.pos_emb = nn.Embedding(max_len, embed_dim)
        d_k = embed_dim // nhead
        self.layers = nn.ModuleList([
            EncoderLayer(embed_dim, nhead, d_k, dim_ff)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, input_ids, mask=None):
        b, seq = input_ids.size()
        pos = torch.arange(seq, device=input_ids.device).unsqueeze(0).expand(b, -1)
        x = self.tok_emb(input_ids) + self.pos_emb(pos)
        for layer in self.layers:
            x = layer(x, mask)
        x = self.norm(x)
        return x
       


In [64]:
from datasets import load_dataset, concatenate_datasets

def load_imdb(n_samples=100):
    dataset = load_dataset("imdb", split="train")

    # How many samples per class
    n_per_class = n_samples // 2

    # Filter each class
    pos = dataset.filter(lambda x: x["label"] == 1).shuffle(seed=42).select(range(n_per_class))
    neg = dataset.filter(lambda x: x["label"] == 0).shuffle(seed=42).select(range(n_per_class))
    
    # Combine and shuffle
    balanced = concatenate_datasets([pos, neg]).shuffle(seed=42)

    texts = balanced["text"]
    labels = balanced["label"]
    return texts, labels

texts, labels = load_imdb(n_samples=10000)

In [66]:
#params
from datasets import load_dataset
from torch.utils.data import DataLoader, TensorDataset

"""
vocab_size
embed_dim
nhead
num_layers ## encoder layers inside the encoder?
dim_feedforward
max_le ## max len of sequence?

"""

vocab_size = 10000      
embed_dim = 64
nhead = 4
num_layers = 6
dim_feedforward = 256
max_len = 128
num_epochs = 10
# Training

""" 
Model init with params
CrossEntropyloss. 
logits no activation
train 
"""



' \nModel init with params\nCrossEntropyloss. \nlogits no activation\ntrain \n'

Benefits of using multiple encoder blocks; you can train a huge model and drop encodel models to make the model more efficient. it gives the freedom to choose the number of blocks.

The code is prepairing the data for an LLM spec MLM and NSP

pre-training: general and large dataset (i.e., wikipedia or common crawl)

domain: specialized domain (i.e., pubmed, twitter)

fine-tuning: task specific datasets i.e., ones provided in the benchmark. 



Image: 
pretraining: imagenet   (learn edges, tectures, colos, shapes)
domain adpation: RICO (apps design) adapt the learned concepty to the domain
fine-tuning: UI classification (classifying the UIs)
